# Giriş

Bu veri seti, İzmir Ekonomi Üniversitesi'nde **9** farklı deniz ürünü türüne ait segmentasyon ve **sınıflandırma** çalışmaları için toplanmıştır. Her bir tür için **1000** adet **RGB** görüntü ve bu görüntülerin karşılık gelen 1000 adet ground truth (gerçek etiket) görüntüsü bulunur. Görüntüler, iki farklı kamera ile toplanmış ve boyutları **590x445** olacak şekilde yeniden boyutlandırılmıştır. Veri seti, özellik çıkarımı, segmentasyon ve sınıflandırma algoritmalarını test etmek ve karşılaştırmak amacıyla kullanılmıştır. Biz ise sınıflandırma için kullanacağız. bir sınıf için görüntüler sıralıdır ve "00000.png" ile "01000.png" arasında adlandırılmıştır.

Şimdi ise hiperparametreler üzerinden belli başlı işlemler yapılacak.
1. 128 ve 512 gizli katmanlı modeller kıyaslanacak.
2. Dropout değeri "0.5" ve "0.7" yapılıp kıyaslanacak
3. Optimizasyon olarak hali hazırda kullanılan **Adam** algoritması **SGD** ile kıyaslanacak 
4. (giriş_boyutu, 512, 128, kategori_sayısı-9- ) katmanlı model eğitilecek
5. En sonda ise eğitilen tüm modeller kıyaslanacak

İlk önce gerekli kütüphaneleri **içeri aktaralım**. (import)

## Kütüphaneleri İçe Aktarma

In [ ]:
import os
import pandas as pd
from PIL import Image  # Görselleri okumak için gerekli kütüphane
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import torch
import torch.nn as nn
import numpy as np

# Uyarıların gizlenmesi
import warnings
warnings.filterwarnings("ignore")

#Tüm sütunları görebilmek için tabloyu ayarladık.
pd.set_option('display.max_colwidth', 1000)

# Veri Setini ----> Pandas DataFrame

Bu kod, bir dosya sisteminde bulunan deniz mahsulleri görsellerinin dosya yollarını ve etiketlerini toplar ve bu verileri pandas kütüphanesinin DataFrame yapısına çevirir. DataFrame kullanmamızın sebebi, veriyi ön incelemede daha kolay ve rahat bir şekilde ele almamızı sağlamasıdır.


In [ ]:
label = []                    #Deniz mahsülünün hangi etikette olduğunu belirten liste
path = []                     #Deniz mahsül görselinin adres yolu

fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

# Verileri klasörlerden toplayarak her tür için görselleri ve etiketleri alıyoruz
for dir_name, _, filenames in os.walk(fish_dir):
    for filename in filenames:
        if os.path.splitext(filename)[-1] == '.png':          # Görselleri sınıflandıracağımız için sadece .png dosyalarını alıyoruz
            if dir_name.split()[-1] != 'GT':                   # Seegmentasyon değil de sınıflandırma yaptığımız için ground truth etiketlerine ihtiyacımız yok.
                file_path = os.path.join(dir_name, filename)
                label.append(os.path.split(dir_name)[-1])  # Adres yolunun adından deniz mahsülünün etiketini elde ediyoruz.
                path.append(file_path)  # Dosya yolunu kaydediyoruz

# Etiket ve dosya yollarından bir DataFrame oluşturuyoruz
data = pd.DataFrame(columns=['path', 'label'])
data['path'] = path
data['label'] = label

data.head()

# Keşifsel Veri Analizi

Veri setindeki hangi deniz mahsülleri ve kaç tane var inceleyelim. 

* 9 tane kategori var ve her birinden bin tane örnek bulunmaktadır. 

In [ ]:
data["label"].value_counts()

## Boş Değer Kontrolü

Bu kod parçası, path ve label sütunlarında eksik (boş) değer olup olmadığını analiz eder. Eksik değerler veri temizleme sürecinde önemli bir adımdır, çünkü makine öğrenmesi modelleri eksik veri içeren kayıtlarla doğru çalışamayabilir.

In [ ]:
# "path" ve "label" sütunlarında eksik değerleri yazdırıyoruz
print(f"Boş path sayısı: {data['path'].isnull().sum()}")
print(f"Boş label sayısı: {data['label'].isnull().sum()}")

In [ ]:
data.describe()

## Veri Setinin Görselleştirilmesi
Veri setindeki her bir kategoriden en az bir tane deniz mahsulü örneğini seçip görselleştirelim. İlk olarak, her kategoriden bir örnek alınır ve görselleri göstermek için 3x3'lük bir grid yapısı oluşturulur. Her görselin üzerinde, görselin kategorisini gösteren bir başlık bulunur. Bu işlem, veri setindeki her bir kategoriden en az bir tane deniz mahsulünü incelememizi sağlar.

In [ ]:
# Her kategoriden bir tane örnek seçiyoruz
sample_data = data.groupby('label').apply(lambda x: x.iloc[0]).reset_index(drop=True)

# Görselleri görselleştirmek için bir ızgara (grid) oluşturuyoruz
plt.figure(figsize=(12, 12))  # Grafik boyutunu ayarlıyoruz

for idx, row in sample_data.iterrows():
    # Görselleri açıyoruz
    img = Image.open(row['path'])
    
    # 3x3'lük bir grid yapısı oluşturuyoruz
    plt.subplot(3, 3, idx + 1)  # 3 satır, 3 sütun, sırayla her bir görsel
    plt.imshow(img)
    plt.axis('off')  # Eksenleri kapatıyoruz
    plt.title(row['label'])  # Başlık olarak etiketi ekliyoruz

plt.tight_layout()
plt.show()

## Frekansı Görselleştirme

Görselleştirme, veri setinin dengeli bir şekilde dağılmış olup olmadığını anlamak açısından önemlidir. "Veri seti dengeli bir şekilde dağılmış" ifadesi, tüm sınıfların eşit veya yakın sayılarda örneklere sahip olduğunu gösterir, bu da sınıflandırma modelleri için önemlidir çünkü dengesiz bir veri seti modelin bazı sınıflarda kötü performans göstermesine yol açabilir.

In [ ]:
# Frekansları saymak için:
label_counts = data["label"].value_counts()

# 1. Sütun Grafiği (Bar Plot)
plt.figure(figsize=(8, 6))
label_counts.plot(kind='bar', color='skyblue')
plt.title('Deniz Mahsülleri Sınıflarının Sütun Grafiği')
plt.xlabel('Label')
plt.ylabel('Frekans')
plt.show()

# 2. Pasta Grafiği (Pie Chart)
plt.figure(figsize=(8, 6))
label_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors)
plt.title('Deniz Mahsülleri Sınıflarının Pasta Grafiği')
plt.ylabel('')  # Y eksenini kaldırıyoruz
plt.show()

# Veri Setini Ayırma

Modelimizin örüntüleri öğrenebilmesi için, veri setini eğitim, doğrulama ve test kümelerine her bir kategoriden eşit sayıda örnek olacak şekilde bölmemiz gerekiyor. Bunu yaparken:

1. Eğitim, doğrulama ve test kümeleri, sırasıyla 8:1:1 oranında ayrılacak.
2. Veri seti karıştırılacak (shuffle) ve böylece dağılımın rastgele olmasını sağlayacağız.
3. Bu oranları dikkate alarak, her bir deniz mahsulü kategorisini eğitim, doğrulama ve test kümelerine ayıracağız.

Kodda **shuffle** fonksiyonu, verilerin karışmasını sağlayarak modelin öğrenme sırasında veri sıralamasından etkilenmesini önler

In [ ]:
train_ratio, validation_ratio, test_ratio = 0.8 , 0.1 , 0.1

# Shuffle edilen DataFrame
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Boş bir 'split' sütunu oluşturuyoruz
shuffled_data['split'] = np.nan

# Kategoriler üzerinde döngü
for category in shuffled_data['label'].unique():
    # Her kategoriye ait verileri seçiyoruz
    category_data = shuffled_data[shuffled_data['label'] == category]
    
    # Kategoriyi shuffle ediyoruz
    category_data = category_data.sample(frac=1, random_state=42)
    
    # Her kategori için oranlara göre ayırıyoruz (%70 train, %15 val, %15 test)
    train_size = int(train_ratio * len(category_data))
    val_size = int(validation_ratio * len(category_data))
    
    # Kategorinin ilk %70'ini 'train', sonraki %15'ini 'val', kalanını 'test' olarak etiketliyoruz
    shuffled_data.loc[category_data.index[:train_size], 'split'] = 'train'
    shuffled_data.loc[category_data.index[train_size:train_size + val_size], 'split'] = 'val'
    shuffled_data.loc[category_data.index[train_size + val_size:], 'split'] = 'test'

# Boş kalan satırları kontrol ediyoruz
empty_rows = shuffled_data[shuffled_data['split'].isna()]
print(f"Boş kalan satır sayısı: {len(empty_rows)}")

# Sonuçları inceleyelim
print(shuffled_data['split'].value_counts())  # Kaç tane train, val, test var görelim
print(shuffled_data.head())  # İlk birkaç satırı kontrol edelim

# CustomDataset Sınıfı

* Bu kod parçası, PyTorch'un veri işleme altyapısında kullanılan **Dataset** sınıfından türetilen CustomDataset sınıfını tanımlar ve görsellerin dosya yollarını ve etiketlerini alarak gerektiğinde veri dönüşümleri uygulamayı sağlar. 
* Bu sınıf, üç temel yöntem olan __init__, __len__ ve __getitem__ fonksiyonları ile veri setine esnek ve kolay erişim sunar.
* **Transform** parametresi, her veri noktasına özel dönüşüm uygulamayı sağlayarak boyutlandırma, normalize etme gibi veri ön işleme işlemlerini kolaylaştırır. Bu yapı, çeşitli veri setlerini (örneğin görseller) işlemek için esneklik sunar
*  Veri yükleme sırasında modelin ihtiyaç duyduğu her bir veri örneğine __getitem__ fonksiyonu aracılığıyla erişim sağlar.

In [ ]:
# Custom Dataset Sınıfı
class CustomDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths  # Görsel dosya yolları
        self.labels = labels          # Etiketler
        self.transform = transform    # Dönüştürme işlemleri

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]  # Görsel dosya yolu
        label = self.labels[idx]         # Etiket
        
        # Görseli açıyoruz
        image = Image.open(img_path).convert("RGB")
        
        # Transform işlemi uygulanırsa
        if self.transform:
            image = self.transform(image)
        
        return image, label

# One Hot Encoding

* Makine öğrenmesi modelleri, kategorik verilerle doğrudan çalışamadığından, bu tür verilerin sayısal formatlara dönüştürülmesi gerekir.
*  **pd.factorize()** kullanarak etiketleri sayısallaştırmak, modelin bu verilerle işlemler yapmasına olanak tanır ve böylece sınıflandırma problemlerinde verinin işlenmesi daha kolay hale gelir. 

* Bu yöntem aynı zamanda verinin kolayca geri dönüştürülebilir (sayısal etiketten orijinal etikete) olmasını sağlar.

In [ ]:
shuffled_data['label_encoded'], label_mapping = pd.factorize(shuffled_data['label'])

In [ ]:
shuffled_data.head(), label_mapping

# DataFrame ---> List

Bu kod parçası, DataFrame yapısında bulunan train, validation ve test kümelerine ait dosya yolları ve etiketleri, iteratif işlemler yapabilmek için liste formatına dönüştürür.

In [ ]:
# Train, Validation ve Test dosya yolları ve etiketlerini DataFrame'den çekiyoruz
train_file_paths = shuffled_data[shuffled_data['split'] == 'train']['path'].tolist()
train_labels = shuffled_data[shuffled_data['split'] == 'train']['label_encoded'].tolist()

val_file_paths = shuffled_data[shuffled_data['split'] == 'val']['path'].tolist()
val_labels = shuffled_data[shuffled_data['split'] == 'val']['label_encoded'].tolist()

test_file_paths = shuffled_data[shuffled_data['split'] == 'test']['path'].tolist()
test_labels = shuffled_data[shuffled_data['split'] == 'test']['label_encoded'].tolist()

# Hiperparametreler

Bu hiperparametreler, görüntü sınıflandırma problemine uygun olarak ayarlanmıştır. 
* **input_size** 64x64 piksel boyutundaki RGB resimlerden gelen 12,288 giriş biriminden oluşur, bu boyut hem modelin hızlı çalışmasını sağlar hem de yeterli detay sunar. 
* **output_size**, veri setindeki 9 farklı deniz mahsulü sınıfını temsil eder ve modelin bu sınıflardan birini tahmin etmesi beklenir.
* **epochs** ise modelin eğitim sürecinde veri seti üzerinde 15 kez geçiş yapacağını ifade eder, bu sayede model yeterli öğrenmeyi sağlayacak şekilde eğitilir.

In [ ]:
input_size = 64 * 64 * 3  # 64x64 boyutunda ve 3 kanallı (RGB) bir resim
output_size = 9  # 9 sınıflı bir sınıflandırma problemi
epochs = 15

# CustomDataset Nesneleri

Bu kod parçasında, tüm görsellerin boyutu önce 590x445 olarak normalize edilmiştir, ancak yeterli donanım kaynağı olmadığı için işlem gücünü optimize etmek amacıyla boyut 64x64 olarak ayarlanmıştır. Görsellerin yeniden boyutlandırılması transforms.Resize((64, 64)) ile yapılır, ardından görseller ToTensor() ile tensöre dönüştürülür. Son olarak, her pikselin değerleri Normalize() fonksiyonu ile [0.5, 0.5, 0.5] ortalamaları ve [0.5, 0.5, 0.5] standart sapmaları kullanılarak normalize edilir. Bu dönüşümler, modelin daha verimli çalışmasını sağlamak için uygulanır.

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dataset'leri oluşturma
train_dataset = CustomDataset(train_file_paths, train_labels, transform=transform)
val_dataset = CustomDataset(val_file_paths, val_labels, transform=transform)
test_dataset = CustomDataset(test_file_paths, test_labels, transform=transform)

# DataLoader

**DataLoader** fonksiyonu, PyTorch'un veri işleme süreçlerinde kullanılan ve veri kümelerini daha verimli bir şekilde işlemenizi sağlayan bir araçtır. DataLoader, veri kümesini **batch** adı verilen **küçük parçalara** bölerek modele aktarır, böylece modelin aynı anda **büyük** bir veri kümesini **bellekte** tutmasına gerek **kalmaz**. Ayrıca, veri yüklemesini paralel olarak yaparak işlem hızını artırır ve her eğitim döngüsünde verilerin **rastgele karıştırılmasını** (shuffle) sağlayarak modelin **aşırı öğrenmesini** önler.

DataLoader ile **CustomDataset sınıfı** arasındaki ilişki ise şu şekildedir: CustomDataset, verilerin nasıl yükleneceğini ve ön işlemlerden geçeceğini belirlerken, DataLoader bu veri kümesini iteratif olarak işler ve modele beslenmek üzere hazırlar. 

In [ ]:
# DataLoader ile veri yükleme
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Kontrol etmek için sadece 1 tane veriye bakalım
for images, labels in train_loader:
    print(f"Görüntülerin şekli: {images.shape}")
    print(f"Etiketlerin şekli: {labels.shape}")
    break

# Sinir Ağı Oluşturma Fonksiyon

Bu fonksiyon, birden fazla modeli esnek bir şekilde oluşturup karşılaştırmak için yazılmıştır. Farklı sayıda gizli katmana sahip modellerin kolayca tanımlanmasını ve aynı yapı üzerinden çeşitli parametrelerle model oluşturulmasını sağlar. Özellikle, input_size, output_size, hidden_layers ve dropout_rate gibi hiperparametreler dinamik olarak değiştirilebilir, böylece kullanıcı farklı yapıdaki modelleri hızlıca deneyebilir. Fonksiyonda kullanılan:

* **Flatten** katmanı, görsel veriyi düzleştirerek 1D vektöre dönüştürür; Linear katmanlar, nöronlar arasında tam bağlantılı sinir ağı kurarak bilgiyi aktarır. 
* **ReLU** aktivasyon fonksiyonu, doğrusal olmayan bir ilişki ekleyerek modelin daha karmaşık örüntüleri öğrenmesini sağlar.
* **Dropout** katmanı, aşırı öğrenmeyi (overfitting) önlemek için rastgele nöronları devre dışı bırakır.
* **Softmax** katmanı, çıkışları olasılık dağılımına dönüştürerek çok sınıflı sınıflandırma problemlerinde her sınıfa ait olasılık tahmini yapar.

Bu fonksiyon sayesinde, manuel model tanımlamalarına gerek kalmadan farklı katman yapıları ve dropout oranları ile modeller oluşturulabilir, eğitilebilir ve karşılaştırılabilir.

In [ ]:
# create_model fonksiyonu: dinamik olarak Sequential yapı oluşturur, Dropout ve Flatten eklenmiş
def create_model(input_size, output_size, hidden_layers, dropout_rate=0.5):
    layers = []
    
    # Flatten katmanını ekle (görsel veriyi 1D vektöre dönüştürmek için)
    layers.append(nn.Flatten())
    
    # Giriş katmanını ekle (input_size -> hidden_layers[0])
    layers.append(nn.Linear(input_size, hidden_layers[0]))
    layers.append(nn.ReLU())  # Aktivasyon fonksiyonu
    layers.append(nn.Dropout(dropout_rate))  # Dropout
    
    # Gizli katmanları ekle
    for i in range(len(hidden_layers) - 1):
        layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
        layers.append(nn.ReLU())  # Her katmana ReLU ekle
        layers.append(nn.Dropout(dropout_rate))  # Dropout
    
    # Çıkış katmanını ekle (hidden_layers[-1] -> output_size)
    layers.append(nn.Linear(hidden_layers[-1], output_size))
    
    # Çıkış için Softmax fonksiyonunu ekle
    layers.append(nn.Softmax(dim=1))
    
    # Sequential modeli oluştur ve döndür
    return nn.Sequential(*layers)

# Doğrulama Fonksiyonu

Bu fonksiyon, modelin **doğrulama kümesi** üzerindeki performansını değerlendirmek için kullanılır. **validate** fonksiyonu, doğrulama verileriyle modeli test ederken gradyan hesaplamayı kapatır, doğrulama kaybını (loss) ve doğruluk oranını (accuracy) hesaplayıp geri döndürür.

In [ ]:
def validate(model, val_loader, criterion, device):
    model.eval()  # Modeli doğrulama moduna geçiriyoruz
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Gradyan hesaplamaya gerek yok
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return val_loss / len(val_loader), accuracy


# Eğitim İçin Fonksiyonlar

Bu iki fonksiyon, modelin eğitim ve doğrulama süreçlerini yönetmek için kullanılır.
* **train_one_epoch** fonksiyonu, modelin bir epoch boyunca eğitim verisi üzerinde eğitilmesini sağlar.
*  **train_model** fonksiyonu ise bu süreci birden fazla epoch boyunca tekrarlar,

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()  # Modeli eğitim moduna geçiriyoruz
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Gradyanları sıfırla
        outputs = model(images)  # İleri geçiş (forward pass)
        loss = criterion(outputs, labels)  # Kayıp hesapla
        loss.backward()  # Geri yayılım (backward pass)
        optimizer.step()  # Optimizasyonu uygula

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)  # Tahmin edilen sınıf
        total += labels.size(0)  # Toplam örnek sayısı
        correct += (predicted == labels).sum().item()  # Doğru tahmin sayısı

    # Eğitim seti üzerindeki doğruluğu hesaplıyoruz
    accuracy = 100 * correct / total

    # Ortalama eğitim kaybı ve doğruluk döndürülüyor
    return running_loss / len(train_loader), accuracy


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    train_loss_arr = []
    train_accurracy_arr = []
    val_loss_arr = []
    val_accuracy_arr = []
    for epoch in range(num_epochs):
        train_loss, accuracy = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_accuracy = validate(model, val_loader, criterion, device)
        
        train_loss_arr.append(train_loss)
        train_accurracy_arr.append(accuracy)
        val_loss_arr.append(val_loss)
        val_accuracy_arr.append(val_accuracy)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, "
              f"Validation Loss: {val_loss:.4f}, "
              f"Validation Accuracy: {val_accuracy:.2f}%")
    return train_loss_arr, train_accurracy_arr, val_loss_arr, val_accuracy_arr

# Test Fonksiyonu

Bu fonksiyon, modelin test verisi üzerindeki performansını değerlendirmek için kullanılır. Test sırasında modelin tahminleri ile gerçek etiketler toplanır ve test işlemi gerçekleştirilir. Sonuç olarak, gerçek ve tahmin edilen etiketler döndürülerek modelin başarımı analiz edilir.

In [ ]:
# Eval fonksiyonu modelin test performansını ölçer
def test_model(model, data_loader, criterion, device):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.view(images.size(0), -1).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())  # Gerçek etiketler
            all_predictions.extend(predicted.cpu().numpy())  # Tahmin edilenler

    return all_labels, all_predictions

# Başarı Metrikleri

Accuracy, Recall, Precision ve F1-Score, sınıflandırma modelinin performansını değerlendiren temel metriklerdir. 
* **Accuracy**, modelin doğru tahmin ettiği örneklerin tüm örnekler içerisindeki oranını gösterir ve genel performansı ölçer, ancak sınıf dengesizliği olduğunda yanıltıcı olabilir.
* **Recall**, modelin pozitif sınıfları ne kadar iyi yakaladığını, yani gerçek pozitiflerin ne kadarının doğru tahmin edildiğini ölçer.
* **Precision**, pozitif olarak tahmin edilen örneklerin gerçekten pozitif olma oranını gösterir ve modelin yanlış pozitifleri azaltma başarısını değerlendirir.
* **F1-Score**, Recall ve Precision'ın harmonik ortalamasıdır ve bu iki metrik arasında denge sağlamak için kullanılır, özellikle dengesiz veri setlerinde faydalıdır.


Bu metrikler bir arada kullanılarak modelin sınıflandırma performansı daha detaylı bir şekilde anlaşılabilir.

In [ ]:
# Tüm modellerin çıktıları dataframe yapısında saklanacak.
# Ondan all_result diye değişken tanımlandı
all_result = pd.DataFrame(columns=['Model', 'Accuracy', 'Recall', 'Precision', 'F1-Score'])

In [ ]:
def metrics(model_name, model, data_loader, criterion, device, class_names):
    global all_result  # Global DataFrame'i kullanıyoruz
    
    # Eval fonksiyonu ile gerçek ve tahmin edilen etiketleri alıyoruz
    true_labels, predicted_labels = test_model(model, data_loader, criterion, device)

    # Classification Report oluşturma
    report = classification_report(true_labels, predicted_labels, target_names=class_names, output_dict=True)
    
    # Confusion Matrix oluşturma
    cm = confusion_matrix(true_labels, predicted_labels)
    
    # Confusion Matrix'i görselleştirme
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.show()
    
    # Sonuçları DataFrame'e eklemek için dictionary oluşturuyoruz
    result = pd.DataFrame([{
        'Model': model_name,
        'Accuracy': report['accuracy'],
        'Recall': report['weighted avg']['recall'],
        'Precision': report['weighted avg']['precision'],
        'F1-Score': report['weighted avg']['f1-score']
    }])
    
    # Global DataFrame'e sonuçları concat ile ekliyoruz
    all_result = pd.concat([all_result, result], ignore_index=True)
    
    # Sonuçları ekrana yazdırıyoruz
    print(f"Model: {model_name}")
    print(f"Accuracy: {report['accuracy']}")
    print(f"Recall: {report['weighted avg']['recall']}")
    print(f"Precision: {report['weighted avg']['precision']}")
    print(f"F1-Score: {report['weighted avg']['f1-score']}")

# Görselleştirme Fonksiyonu (Loss ve Acc)

Doğrulama ve eğitim sürecindeki loss ve accuracy değerlerini daha iyi değerlendirmek için yardımcı fonksiyon.

In [ ]:
def plot_training_curves(train_loss, val_loss, train_accuracy, val_accuracy):
    epochs = range(1, len(train_loss) + 1)

    # Loss grafiği
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss, 'b', label='Train Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Loss per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy grafiği
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracy, 'b', label='Train Accuracy')
    plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# 128 Katmanlı Model


In [ ]:
hidden_layers = [128]  # Gizli katmanlar
dropout_rate = 0.5  # Dropout oranı

# create_model fonksiyonu ile modeli oluştur
model_128 = create_model(input_size, output_size, hidden_layers, dropout_rate)

# Modeli kontrol etmek için yapı bilgilerini yazdır
print(model_128)

In [ ]:
model_128 = model_128.to(device)

# Kayıp fonksiyonu ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer_128 = torch.optim.Adam(model_128.parameters(), lr=0.001)

# 128 nöronlu modelin eğitim aşaması
train_loss_arr_128, train_accuracy_arr_128, val_loss_arr_128, val_accuracy_arr_128 = train_model(
    model_128, train_loader, val_loader, criterion, optimizer_128, num_epochs=epochs, device=device)

# Eğitim sonuçlarını görselleştirme
plot_training_curves(train_loss_arr_128, val_loss_arr_128, train_accuracy_arr_128, val_accuracy_arr_128)

# 128 nöronlu modelin sonuçlarını hesaplayalım
metrics("model_128", model_128, test_loader, criterion, device, label_mapping)

# 512 Katmanlı Model

In [ ]:
hidden_layers = [512]  # Gizli katmanlar
dropout_rate = 0.5  # Dropout oranı

# create_model fonksiyonu ile modeli oluştur
model_512 = create_model(input_size, output_size, hidden_layers, dropout_rate)

# Modeli kontrol etmek için yapı bilgilerini yazdır
print(model_512)

In [ ]:
model_512 = model_512.to(device)

# Kayıp fonksiyonu ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer_512 = torch.optim.Adam(model_512.parameters(), lr=0.001)

# 512 nöronlu modelin eğitim aşaması
train_loss_arr_512, train_accuracy_arr_512, val_loss_arr_512, val_accuracy_arr_512 = train_model(
    model_512, train_loader, val_loader, criterion, optimizer_512, num_epochs=epochs, device=device)

# Eğitim sonuçlarını görselleştirme
plot_training_curves(train_loss_arr_512, val_loss_arr_512, train_accuracy_arr_512, val_accuracy_arr_512)
metrics("model_512", model_512, test_loader, criterion, device, label_mapping)

# 0.7 Dropout Model

In [ ]:
# 128 Nöronlu Model (Dropout 0.7)
hidden_layers = [128]  # Gizli katmanlar
dropout_rate = 0.7  # Dropout oranı

# create_model fonksiyonu ile modeli oluştur
model_128_dropout_07 = create_model(input_size, output_size, hidden_layers, dropout_rate)

# Modeli cihaza aktar
model_128_dropout_07 = model_128_dropout_07.to(device)

# Kayıp fonksiyonu ve optimizer tanımlama
optimizer_128_dropout_07 = torch.optim.Adam(model_128_dropout_07.parameters(), lr=0.001)

# 128 nöronlu modelin eğitim aşaması (dropout 0.7)
train_loss_arr_128_07, train_accuracy_arr_128_07, val_loss_arr_128_07, val_accuracy_arr_128_07 = train_model(
    model_128_dropout_07, train_loader, val_loader, criterion, optimizer_128_dropout_07, num_epochs=epochs, device=device)

# Eğitim sonuçlarını görselleştirme
plot_training_curves(train_loss_arr_128_07, val_loss_arr_128_07, train_accuracy_arr_128_07, val_accuracy_arr_128_07)
metrics("model_128_dropout_07", model_128_dropout_07, test_loader, criterion, device, label_mapping)

# SGD Algoritmalı Model

In [ ]:
# 128 Nöronlu SGD Model
hidden_layers = [128]  # Gizli katmanlar
dropout_rate = 0.5  # Dropout oranı

# create_model fonksiyonu ile modeli oluştur
model_128_sgd = create_model(input_size, output_size, hidden_layers, dropout_rate)

# Modeli cihaza aktar
model_128_sgd = model_128_sgd.to(device)

# Kayıp fonksiyonu ve optimizer tanımlama (SGD)
optimizer_128_sgd = torch.optim.SGD(model_128_sgd.parameters(), lr=0.01, momentum=0.9, nesterov=True)

# 128 nöronlu modelin eğitim aşaması (SGD ile)
train_loss_arr_128_sgd, train_accuracy_arr_128_sgd, val_loss_arr_128_sgd, val_accuracy_arr_128_sgd = train_model(
    model_128_sgd, train_loader, val_loader, criterion, optimizer_128_sgd, num_epochs=epochs, device=device)

# Eğitim sonuçlarını görselleştirme
plot_training_curves(train_loss_arr_128_sgd, val_loss_arr_128_sgd, train_accuracy_arr_128_sgd, val_accuracy_arr_128_sgd)
metrics("model_128_sgd", model_128_sgd, test_loader, criterion, device, label_mapping)


# [128,512] Gizli Katmanlı Model

In [ ]:
# 512 ve 128 Nöronlu Gizli Katmanlar
hidden_layers = [128, 512]  # Gizli katmanlar
dropout_rate = 0.5  # Dropout oranı

# create_model fonksiyonu ile modeli oluştur
model_512_128 = create_model(input_size, output_size, hidden_layers, dropout_rate)

# Modeli cihaza aktar
model_512_128 = model_512_128.to(device)

# Kayıp fonksiyonu ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer_512_128 = torch.optim.SGD(model_512_128.parameters(), lr=0.01, momentum=0.9, nesterov=True)

# Modelin eğitim aşaması
train_loss_arr_512_128, train_accuracy_arr_512_128, val_loss_arr_512_128, val_accuracy_arr_512_128 = train_model(
    model_512_128, train_loader, val_loader, criterion, optimizer_512_128, num_epochs=epochs, device=device)



In [ ]:
# Eğitim sonuçlarını görselleştirme
plot_training_curves(train_loss_arr_512_128, val_loss_arr_512_128, train_accuracy_arr_512_128, val_accuracy_arr_512_128)
metrics("model_128_512", model_512_128, test_loader, criterion, device, label_mapping)

# Model Sonuçları

In [ ]:
all_result